# Homework 4

In [25]:
import datetime
import json
import boto3
import requests

## Fetch the top viewed pages for a specific date

In [26]:
# Try different dates to see how the data changes
DATE_PARAM = "2025-11-27"
date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")
# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")
# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text
print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")
# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/27
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"27","articles":[{"article":"Main_Page","views":5988101,"rank":1},{"article":"Special:Search","views":797249,"rank":2},{"article":"Stranger_Things_season_5","views":655701,"rank":3},{"article":"Stranger_Things","views":353714,"rank":4},{"article":"Jack_White","views":349384,"rank":5},{"article":"Google_Chrome","views":306639,"rank":6},{"article":"Wikipedia:Featured_pictures","views":248728,"rank":7},{"arti...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 55549


## Transform the response to JSON Lines format with these fields

In [27]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_edits = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_edits[:6]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_edits)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 5988101, "rank": 1, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:54:27.285172"}
{"title": "Special:Search", "views": 797249, "rank": 2, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:54:27.285172"}
{"title": "Stranger_Things_season_5", "views": 655701, "rank": 3, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:54:27.285172"}
{"title": "Stranger_Things", "views": 353714, "rank": 4, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:54:27.285172...


## Upload to S3 at `raw-views/raw-views-YYYY-MM-DD.json`

In [28]:
S3_WIKI_BUCKET = "andrea-wikidata"
s3 = boto3.client("s3")
s3_key = f"raw-views/raw-views-{date.strftime("%Y-%m-%d")}.json"

s3.put_object(
    Bucket=S3_WIKI_BUCKET,
    Key=s3_key,
    Body=json_lines,
)
print(f"Uplodaded data to s3://{S3_WIKI_BUCKET}/{s3_key}")

Uplodaded data to s3://andrea-wikidata/raw-views/raw-views-2025-11-27.json


## Re-execute the notebook twice more, setting the date to different days
done